### Programers 대장균들의 자식 수 구하기.

이 문제는 ID column 과 같은 값이 있는 PARENT_ID 를 찾고 개수를 카운팅하는 문제.

### 하면 안되는 생각의 궤적.

```python
select ID,
       count(PARENT_ID)
       
from ECOLI_DATA

group by PARENT_ID
```

위와 같은 방향으로 작성하는 경우, 

`GROUP BY 절에 속한 열 이외의 열을 집계 함수 없이 SELECT 절에 포함할 수 없기 때문에 clause 오류가 발생한다`

다시 말해, ID 는 group by 에 aggregate function 없이 단독으로 쓰였기 때문에 구문 오류가 발생한다.

그렇기 때문에 sub-query 를 작성하여 원하는 값을 출력하도록 하던가, 그 외의 방법을 사용해야함.

### 레코드 적고, 가끔 [sub - query]

```python
select ID,
      (select count(ID) from ECOLI_DATA where PARENT_ID = main.ID) as child_count 
from ECOLI_DATA as main
order by ID asc
```

### `where PARENT_ID = ID) as child_count` 이 안되는 이유.

where PARENT_ID = main.ID 은 외부의 main 테이블에서 동작하기 때문에 우리가 원하는 결과를 얻을 수 있고,

where PARENT_ID = ID 의 경우에는 `같은 row 의 ID 와 PARENT_ID 에 해당하는 ID 를 세기` 때문에 결과가 다르고 의도하는 방향이 아니다.

(select count(ID) from ECOLI_DATA where PARENT_ID = main.ID) 이 쿼리의 결과로는 ID 별로 계산된 값들이 한 개씩 출력될 것.


### 레코드 많고, 빈번

```python
with cte as(
    select  PARENT_ID as ID,
            count(*) as CHILD_COUNT
    from    ECOLI_DATA
    group by PARENT_ID)

select  main.ID,
        coalesce(cte.CHILD_COUNT,0) as CHILD_COUNT
from    cte
right join ECOLI_DATA as main on cte.ID = main.ID
order by main.ID asc;
```

우선 group by 외의 절을 select 절에서 단독으로 쓸 수 없다는 것을 인지하고, 

구하는 것이 자식의 개수가 몇 개냐? 이기 때문에 

존재하고 있는 parent_id 를 그룹화 하여 count(*) 를 하면 

그룹화 된 parent_id 가 각각의 값들에 대한 개별 집계하게 된다.

```python
with cte as(
    select  PARENT_ID as ID,
            count(*) as CHILD_COUNT
    from    ECOLI_DATA
    group by PARENT_ID)
```

`아래의 표는 위 쿼리의 결과이다.`

| ID   | CHILD_COUNT |
|------|-------------|
| 1    | 1           | 
| 2    | 2           | 
| NULL | NULL        | 
| 4    | 1           | 
| NULL | NULL        | 
| NULL | NULL        |

위와 같이 작성되고, 어차피 ID 는 그룹화 하기 위한 용도였고 필요한 것은 개별 항목들의 카운팅이다.

null 값은 0 으로 표현하기 위해서 coalesce 를 써주고 right join 을 작성하면 된다. 

### right join 만 된 상태

| ID   | CHILD_COUNT | ID | PARENT_ID | SIZE_OF_COLONY | DIFFERENTIATION_DATE | GENOTYPE |
|------|-------------|----|-----------|----------------|----------------------|----------|
| 1    | 1           | 1  | NULL      | 10             | 2019-01-01           | 5        |
| 2    | 2           | 2  | NULL      | 2              | 2019-01-01           | 3        |
| NULL | NULL        | 3  | 1         | 100            | 2020-01-01           | 4        |
| 4    | 1           | 4  | 2         | 17             | 2020-01-01           | 4        |
| NULL | NULL        | 5  | 2         | 10             | 2020-01-01           | 6        |
| NULL | NULL        | 6  | 4         | 101            | 2021-01-01           | 22       |

여기서 ID, CHILD_COUNT (null 값은 0 으로) 를 구하면 

```python
select  main.ID,
        coalesce(cte.CHILD_COUNT,0) as CHILD_COUNT
from    cte
right join ECOLI_DATA as main on cte.ID = main.ID
order by main.ID asc;
```
 
위와 같이 작성할 수 있다.

### 레코드가 많고 조회가 빈번할 때, 위의 쿼리의 장점.

with 절로 데이터를 미리 계산하여 재사용

### 최종 쿼리 결과

| ID | CHILD_COUNT |
|----|-------------|
| 1  | 1           |
| 2  | 2           |
| 3  | 0           |
| 4  | 1           |
| 5  | 0           |
| 6  | 0           |

배울 것, 생각할 것이 많은 문제였음.